In [1]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Define dataset directories

In [2]:
fruit_dir = '/kaggle/input/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset(12000)/Fruits'
veg_dir = '/kaggle/input/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset(12000)/Vegetables'


# Define fruit and vegetable classes

In [3]:

fruit_classes = ['FreshApple', 'RottenApple', 'FreshBanana', 'RottenBanana', 'FreshMango', 'RottenMango', 'FreshOrange', 'RottenOrange', 'FreshStrawberry', 'RottenStrawberry']
veg_classes = ['FreshCarrot', 'RottenCarrot', 'FreshTomato', 'RottenTomato', 'FreshCucumber', 'RottenCucumber', 'FreshPotato', 'RottenPotato', 'FreshBellpepper', 'RottenBellpepper']



# Define a custom dataset class

In [4]:

class CustomDataset(Dataset):
    def __init__(self, data_dir, classes, transform=None):
        self.data_dir = data_dir
        self.classes = classes
        self.transform = transform
        self.images = []
        self.labels = []

        for class_name in classes:
            class_path = os.path.join(data_dir, class_name)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img = Image.open(img_path).convert('RGB')
                self.images.append(img)
                self.labels.append(classes.index(class_name))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)

        return image, label



# Define transforms

In [5]:

data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])




# Create datasets and data loaders

In [6]:

fruit_dataset = CustomDataset(fruit_dir, fruit_classes, transform=data_transform)
veg_dataset = CustomDataset(veg_dir, veg_classes, transform=data_transform)


/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


# Combine datasets

In [7]:

combined_dataset = torch.utils.data.ConcatDataset([fruit_dataset, veg_dataset])



# Define train and test split

In [8]:

train_size = int(0.8 * len(combined_dataset))
test_size = len(combined_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(combined_dataset, [train_size, test_size])



# Create data loaders

In [9]:

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Define the model

In [10]:

resnet = models.resnet50(pretrained=True)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 152MB/s]


# Freeze all layers except the final fully connected layer

In [11]:

for param in resnet.parameters():
    param.requires_grad = False

# Replace the final fully connected layer

In [12]:

num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, len(fruit_classes) + len(veg_classes))



# Define loss function and optimizer

In [13]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)


# Train the model

In [14]:


num_epochs = 10
resnet.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}")


Epoch [1/10], Loss: 0.8594485374291738
Epoch [2/10], Loss: 0.41755169093608857
Epoch [3/10], Loss: 0.34463432284692924
Epoch [4/10], Loss: 0.3091974190125863
Epoch [5/10], Loss: 0.2747807362427314
Epoch [6/10], Loss: 0.24900790157417457
Epoch [7/10], Loss: 0.23927819456905128
Epoch [8/10], Loss: 0.22488371507575114
Epoch [9/10], Loss: 0.2210466382900874
Epoch [10/10], Loss: 0.21152354178329308


# Evaluate the model

In [15]:

resnet.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = resnet(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9275
